<h1>Установка и импорт нужных пакетов</h1>

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd

In [4]:
cycle_time_table = pd.read_excel(f"/content/drive/MyDrive/Safia_planning/input_master_data_safia.xlsx", sheet_name='cycle_time_table')
time_mode = pd.read_excel(f"/content/drive/MyDrive/Safia_planning/input_master_data_safia.xlsx", sheet_name='time_mode')
cream_data = pd.read_excel(f"/content/drive/MyDrive/Safia_planning/input_master_data_safia.xlsx", sheet_name='cream_data')
current_plan = pd.read_excel(f"/content/drive/MyDrive/Safia_planning/safia_plan.xlsx", sheet_name='current_date')
cat_values = time_mode['start']
time_mode['start'] = pd.Categorical(time_mode['start'], categories=cat_values, ordered=True)

In [5]:
# Объединяем таблицы с циклами и планом
operation_plan_for_all_calls = current_plan.merge(cycle_time_table, on=['sku', 'operation'], how='left')
operation_plan_for_all_calls['production_duration_sec'] = operation_plan_for_all_calls['quantity'] * operation_plan_for_all_calls['cycle_time_sec']

# Дополняем таблицу временного режима столбцами временными границами с начала смены
time_mode['duration_cumulative'] = time_mode['duration'].cumsum()
time_mode['duration_cumulative_start'] = pd.Series([0])._append(time_mode['duration_cumulative'].shift(1).iloc[1:]).astype('int')

# Перечисляем все ячейки из плана
cell_list = operation_plan_for_all_calls['cell'].unique()

# Создаем обособленные по ячейкам планы
cell_plan_list = []
for i in cell_list:
  cell_plan_list.append(operation_plan_for_all_calls[operation_plan_for_all_calls['cell'] == i])
for i in cell_plan_list:
  i['cycle_time_sec_cumulative'] = i['production_duration_sec'].cumsum()
  i['duration_cumulative_start'] = pd.Series([0])._append(i['cycle_time_sec_cumulative'].shift(1).iloc[1:]).astype('int')

# Производим основные расчеты через создание посекундки
df_list = []
for i in cell_plan_list:
  df = pd.DataFrame({'Номер секунды': range(1, time_mode['duration_cumulative'].iloc[-1] + 1)})
  def find_time_window(row):
    for j in range(len(time_mode)):
        if time_mode['duration_cumulative_start'][j] <= row['Номер секунды'] <= time_mode['duration_cumulative'][j]:
            return time_mode['start'][j]
    return None
  def find_sku(row):
    for j in i.index:
        if i['duration_cumulative_start'][j] <= row['Номер секунды'] <= i['cycle_time_sec_cumulative'][j]:
            return i['sku'][j]
    return None
  def find_operation(row):
    for j in i.index:
        if i['duration_cumulative_start'][j] <= row['Номер секунды'] <= i['cycle_time_sec_cumulative'][j]:
            return i['operation'][j]
    return None

  df['time_window'] = df.apply(find_time_window, axis=1)
  df['sku'] = df.apply(find_sku, axis=1)
  df['operation'] = df.apply(find_operation, axis=1)
  pivot_table = pd.pivot_table(df, index=['time_window', 'sku', 'operation'], aggfunc='count').reset_index().merge(operation_plan_for_all_calls[['cell', 'sku', 'operation', 'cycle_time_sec']], on=['sku', 'operation'], how='left')
  pivot_table['quantity'] = (pivot_table['Номер секунды'] / pivot_table['cycle_time_sec']).astype('int')
  pivot_table['time_window'] = pd.Categorical(pivot_table['time_window'], categories=cat_values, ordered=True)
  pivot_table = pivot_table.sort_values('time_window')
  df_list.append(pivot_table[['cell', 'time_window', 'sku', 'operation', 'quantity']])

<ipython-input-5-113b08cfcf03>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i['cycle_time_sec_cumulative'] = i['production_duration_sec'].cumsum()
<ipython-input-5-113b08cfcf03>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i['duration_cumulative_start'] = pd.Series([0])._append(i['cycle_time_sec_cumulative'].shift(1).iloc[1:]).astype('int')
<ipython-input-5-113b08cfcf03>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [23]:
df_list_with_cream = []
for i in df_list:
  cake_plan_with_cream = i.merge(cream_data, on=['sku', 'operation'], how='left')
  cake_plan_with_cream['cream_plan'] = cake_plan_with_cream['quantity'] * cake_plan_with_cream['gr']
  df_list_with_cream.append(cake_plan_with_cream)
df_cream_list = []
for i in df_list_with_cream:
  df_by_creams_and_time = pd.pivot_table(i, index=['time_window', 'raw_materials'], values='cream_plan', aggfunc='sum').reset_index()
  df_cream_list.append(df_by_creams_and_time)
merged_df = pd.concat(df_cream_list, axis=0)
cream_time = pd.pivot_table(merged_df, index=['time_window', 'raw_materials'], values='cream_plan', aggfunc='sum').reset_index()
cream_time['time_window'] = pd.Categorical(cream_time['time_window'], categories=cat_values, ordered=True)
cream_time = cream_time.sort_values('time_window')
cream_time = cream_time[cream_time['cream_plan'] != 0]
cream_time

,time_window,raw_materials,cream_plan
0,8:00,Сливочный крем №1,38540.000000
1,8:00,Сливочный крем №2,41246.000000
3,8:00,Сливочный крем №5,49000.000000
14,9:00,сливочный крем №4,17160.000000
13,9:00,сливочный крем №3,1400.000000
9,9:00,Сливочный крем №2,34575.000000
8,9:00,Сливочный крем №1,44550.000000
11,9:00,Сливочный крем №5,42500.000000
21,10:00,сливочный крем №3,16650.000000
22,10:00,сливочный крем №4,54120.000000


In [28]:
i['cell'][0]

'заг 1/2'

In [24]:
@st.cache
def convert_to_csv(df):
    # IMPORTANT: Cache the conversion to prevent computation on every rerun
    return df.to_csv(index=False).encode('utf-8')

csv = convert_to_csv(df)

# display the dataframe on streamlit app
st.write(df)

# download button 1 to download dataframe as csv
download1 = st.download_button(
    label="Download data as CSV",
    data=csv,
    file_name='large_df.csv',
    mime='text/csv'
)

# download button 2 to download dataframe as xlsx
with pd.ExcelWriter(buffer, engine='xlsxwriter') as writer:
    # Write each dataframe to a different worksheet.
    df.to_excel(writer, sheet_name='Sheet1', index=False)

    download2 = st.download_button(
        label="Download data as Excel",
        data=buffer,
        file_name='large_df.xlsx',
        mime='application/vnd.ms-excel'
    )

NameError: ignored